# This Notebook is the project deliverable for my completion of the <span style="text-decoration: underline"> Coursera Applied Data Science </span> course at the end of the Data Scientist Professional certificate program. This notebook will evolve to contain all of the course assignments in labelled sections.

Included in this Notebook are:
1. Initial course assignment
2. Segmenting and Clustering Neighborhoods in Toronto Part 1
3. Segmenting and Clustering Neighborhoods in Toronto Part 2
4. Segmenting and Clustering Neighborhoods in Toronto Part 3
5. Capstone Project - The Battle of Neighborhoods 
   

## This section is the initial course assignment.

In [2]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


### End of initial course assignment.

## This section is <span style="text-decoration: underline">Segmenting and Clustering Neighborhoods in Toronto - Part 1 </span>
    
For this step of the assignment I will create a dataframe as required by the project description.

'Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.' However, the format at the subject wesite was updated so the following site was used to obtain the data in a format already in use in this project: https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=946126446.

In [3]:
raw=pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=935851093',header=0)  # get the data from website and put into table format
df=pd.DataFrame(raw[0])     # take the list entry that corresponds to the website table and make it a dataframe
df.head()                   # look at the contents

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
df1=df.loc[df['Borough'] !='Not assigned']   # create a new dataframe omitting the missing boroughs
df1.reset_index(drop=True,inplace=True)      # reset the dataframe index
df1.columns=["PostalCode","Borough","Neighborhood"]   # adjust Post code columnname
df1.info()                                  # find the range of entries for the next step of cleaning up missing neighborhood names

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 3 columns):
PostalCode      210 non-null object
Borough         210 non-null object
Neighborhood    210 non-null object
dtypes: object(3)
memory usage: 5.0+ KB


In [5]:
df1.head(12)                             # peek for any unassigned neighborhoods

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Not assigned
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [6]:
# run through the dataframe replacing Neighborhood name with Borough name if the neighborhood name is unassigned
for i in range(0,210):
    if df1.loc[i,'Neighborhood']=='Not assigned':
        df1.loc[i,'Neighborhood'] = df1.loc[i,'Borough']
df1.head(12)             # peek at the results 

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [7]:
df1=df1.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index()    # merge neighborhoods with the same borough and postal code, results in dataframe being sorted by postal code and
                                                                                           # neighborhood names being sorted within postal codes
df1.head(12)                                                                            # peek at the results

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [8]:
df1.shape              # check the final shape of the dataframe

(103, 3)

### End of Segmenting and Clustering Neighborhoods in Toronto - Part 1

## This section is <span style="text-decoration: underline"> Segmenting and Clustering Neighborhoods in Toronto - Part 2 </span>

For this part of the assignment I will add location coordinates to the dataframe created in Part 1 of the project. The source of the coordinates is a file located at: http://cocl.us/Geospatial_data

In [9]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data           # get the coordinates file
df2=pd.read_csv("Geospatial_Coordinates.csv")                                     # format the contents in a dataframe
df2.shape                                                                         # check the shape to see if mismatch when compared to neighborhoods dataframe

(103, 3)

In [10]:
# shapes match and both files are sorted in ascending order by postal code so just make sure postal codes match when adding the coordinates
for i in range(0,103):
    if df1.loc[i,"PostalCode"] == df2.loc[i,"Postal Code"]:
        df1.loc[i,"Latitude"] = df2.loc[i,"Latitude"]
        df1.loc[i,"Longitude"] = df2.loc[i,"Longitude"]
df1.head()                                                  # check the results

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


### End of Segmenting and Clustering Neighborhoods in Toronto - Part 2

## This section is <span style="text-decoration: underline"> Segmenting and Clustering Neighborhoods in Toronto - Part 3 </span>

For this part of the assignment we'll look at the distribution of post offices based on how many neighborhoods they serve. This brief exploration is to obtain an insight based on the data already available. Certainly more exploration can be performed using the Foursquare data but this project is kept simple to facilitate moving on to the final project.

In [11]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!pip install folium
import folium

     |████████████████████████████████| 92kB 12.2MB/s eta 0:00:01


In [12]:
# get the toronto coordinates

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [13]:
# create map of the Toronto area 
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add neighborhhod area markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
# set new column with value for number of neighborhoods served by post office
for i in range(0,103):
    df1.loc[i,"Num"]=df1.loc[i,"Neighborhood"].lower().count(',')+1         
df1["Num"]=df1["Num"].astype(int)               # cast the count to integers for use with mapping later on
df1.groupby('Num').count()                      # peek at distribution of post offices serving more than one neighborhood

,PostalCode,Borough,Neighborhood,Latitude,Longitude
Num,,,,,
1,47,47,47,47,47
2,29,29,29,29,29
3,17,17,17,17,17
4,4,4,4,4,4
5,3,3,3,3,3
7,1,1,1,1,1
8,2,2,2,2,2


In [20]:
# map the clusters by unique colors
map_clusters=folium.Map(location=[latitude,longitude],zoom_start=12)
kclusters=8                  # number of clusters from groupby.count
# set color scheme
x=np.arange(kclusters)
ys=[i+x+(i*x)**2 for i in range(kclusters)]
colors_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in colors_array]
#set markers with cluster color scheme
for lat,lon,poi,cluster in zip(df1['Latitude'],df1['Longitude'],df1['Neighborhood'],df1['Num']):
    label=folium.Popup(str(poi)+'Cluster'+str(cluster),parse_html=True)
    folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_toronto)
    
map_toronto

### End of Segmenting and Clustering Neighborhoods in Toronto - Part 3

## This section is <span style="text-decoration: underline"> Capstone Project - The Battle of Neighborhoods </span>

### End of Capstone Project - The Battle of Neighborhoods
### End of Notebook